In [3]:
# Libraries used to train models & manipulate data
import numpy as np
import pandas as pd

# * The number of times an item has to be seen for it to be included
rep_times = 3

### Kaggle import: https://github.com/Kaggle/kaggle-api
# kaggle datasets download -f dump.csv --unzip killbot/linkedin-profiles-and-jobs-data
members = pd.read_csv(r'../dump_cleaned.csv')

# Filter for useful entries 
members = members.filter(items=['memberUrn', 'posTitle'])
members.head(3)

,memberUrn,posTitle
0,10013136,project manager
1,10013136,project manager
2,100186032,senior marketing manager


In [4]:
# Group members by position
members_grouped = members.groupby('posTitle')['memberUrn'].apply(list) # 2817 jobs w/ more than 1 member
# Series -> DataFrame & Reset indices
members_grouped = members_grouped.to_frame()
members_grouped = members_grouped.reset_index()

print(members_grouped.shape)
members_grouped.head(3)

(1458, 2)


,posTitle,memberUrn
0,.net developer,"[100497522, 105013375, 11628672, 142592347, 16..."
1,.net software developer,"[142592347, 48796961, 65374720]"
2,3d generalist,"[245710462, 245710462, 245710462, 5580986]"


In [5]:
#! HORRIBLE MEMORY
ids = members['memberUrn'].unique().tolist()

def multiEncodeMembers(urnList):
    l = [0]*len(ids)

    for urn in urnList:
        if urn:
            index = ids.index(urn)
            l[index] = 1
    return l

# Multi=encode the members
members_grouped['memberUrn'] = members_grouped['memberUrn'].apply(multiEncodeMembers)
# Get rid of entries that only appear once
# members_grouped = members_grouped[members_grouped.posTitle.duplicated(keep=False)] # No values?
# Drop duplicate entries now
# members_grouped['posTitle'] = members_grouped['posTitle'].drop_duplicates()

print(members_grouped.shape)
members_grouped.head(3)

(1458, 2)


,posTitle,memberUrn
0,.net developer,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,.net software developer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3d generalist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
#! HORRIBLE MEMORY
pos_names = members['posTitle'].unique().tolist()

# Multi=encode the members
pos_encoded = members_grouped['posTitle'].apply(lambda x: pos_names.index(x)).to_frame()
pos_encoded = pos_encoded.rename(columns={'posTitle': 'posEncoded'})
# Group data together
train = pd.concat([members_grouped, pos_encoded], axis=1)
train = train.dropna()

train.to_json('w2v_train.json')

print(len(ids))
print(len(pos_names))
print(train.shape)
train.head(5)

5643
1458
(1458, 3)


,posTitle,memberUrn,posEncoded
0,.net developer,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",17
1,.net software developer,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",878
2,3d generalist,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1261
3,academic (part – time),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1265
4,academic tutor,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1095
